In [47]:
import ast

import numpy as np
import pandas as pd

from pysgpp.extensions.datadriven.learner import LearnerBuilder
from pysgpp.extensions.datadriven.uq.plot.plot1d import plotSG1d
from pysgpp.extensions.datadriven.uq.plot.plot2d import plotSG2d

import matplotlib.pyplot as plt

In [48]:
filename = 'output.csv'

df = pd.read_csv(filename, header = None, names = ['params', 'time'])
df.params = df.params.apply(ast.literal_eval)

for param in df.iloc[0].params.keys():
    df[param] = df.params.apply(lambda row: row[param])
    
df['density'] = (df.MOL_X*df.MOL_Y)/(df.DOMAIN_SIZE_X*df.DOMAIN_SIZE_Y)
del df['params']

In [49]:
df.head()

,time,DOMAIN_SIZE_X,TEMP,DOMAIN_SIZE_Y,DELTA_T,V_x,V_y,LINKED_CELL_SIZE_Y,LINKED_CELL_SIZE_X,DOMAIN_SIZE,MOL_X,MOL_Y,CUTOFF_RADIUS,TIMESTEPS,BLOCK_SIZE,density
0,NaN,59,2.5,59,0.002,1,1.5,37.75,38.25,5,85,56,17.25,1000,14.75,1.367423
1,NaN,59,2.5,59,0.002,1,1.5,35.75,36.25,5,89,47,17.50,1000,14.75,1.201666
2,NaN,59,2.5,59,0.002,1,1.5,36.00,36.25,5,65,83,17.25,1000,14.75,1.549842
3,179.755897,59,2.5,59,0.002,1,1.5,36.50,37.75,5,49,51,17.50,1000,14.75,0.717897
4,669.340109,59,2.5,59,0.002,1,1.5,39.00,37.75,5,78,63,17.00,1000,14.75,1.411663


In [53]:
params = ['DOMAIN_SIZE_X', 'DOMAIN_SIZE_Y', 'MOL_X', 'MOL_Y', 'LINKED_CELL_SIZE_X', 'LINKED_CELL_SIZE_Y',
          'BLOCK_SIZE', 'CUTOFF_RADIUS', 'density']

In [66]:
num_samples = df.shape[0]

X = df[params].values
Y = df.time.values

In [88]:
levels = 5 
num_dims = len(params)

builder = LearnerBuilder()
builder.buildRegressor()
builder.withTrainingDataFromNumPyArray(X, Y)
builder = builder.withGrid()
builder.withLevel(levels)
builder = builder.withSpecification()
builder.withLambda(0.0001)

builder.withIdentityOperator()
builder = builder.withStopPolicy()
builder = builder.withCGSolver()
builder.withAccuracy(0.000100)
builder.withImax(500)
learner = builder.andGetResult()

learner.learnData()

<pysgpp.pysgpp_swig.DataVector; proxy of <Swig Object of type 'sgpp::base::DataVector *' at 0x7fd9d6044b40> >

In [74]:
learner.alpha.array()

array([ 0.,  0.,  0., ...,  0.,  0.,  0.])

In [87]:
X

array([[ 59.        ,  59.        ,  85.        ,  56.        ,
         38.25      ,  37.75      ,  14.75      ,  17.25      ,
          1.36742315],
       [ 59.        ,  59.        ,  89.        ,  47.        ,
         36.25      ,  35.75      ,  14.75      ,  17.5       ,
          1.20166619],
       [ 59.        ,  59.        ,  65.        ,  83.        ,
         36.25      ,  36.        ,  14.75      ,  17.25      ,   1.549842  ],
       [ 59.        ,  59.        ,  49.        ,  51.        ,
         37.75      ,  36.5       ,  14.75      ,  17.5       ,
          0.71789716],
       [ 59.        ,  59.        ,  78.        ,  63.        ,
         37.75      ,  39.        ,  14.75      ,  17.        ,
          1.41166332]])